In [27]:
import pandas as pd
import numpy as np
import glob
import nibabel as nib
from nibabel.testing import data_path
import os

In [28]:
df = pd.read_csv("/Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS/Normalized Images/norm_vals.csv")

In [29]:
writepath = "/Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS/Normalized Images"
readpath = "/Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS"

In [30]:
for row,items in df.iterrows():
    example_filename = os.path.join(readpath, items["filename"])
    img = nib.load(example_filename)
    data = img.get_fdata()
    data = data * items["mRatio"]
    img = nib.Nifti1Image(data, img.affine)

    nib.save(img, writepath + "/" + "normalized_" + items["filename"])

In [31]:
cd /Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS

/Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS


In [32]:
filenames = np.array([])
sampleID = np.array([])
for file in glob.glob("*.gz"):
    if file[-13:] == "labels.nii.gz":
        split = file.split('_')[0:3]
        sampleID = np.append(sampleID,( "_".join(split)))
        filenames = np.append(filenames,file)

In [33]:
labelsDF = pd.DataFrame({"id": sampleID, "labelFile": filenames})

In [34]:
df['id'] = list(map(lambda x: "_".join(x.split('_')[0:3]), df['id']))

In [36]:
merge = df.merge(labelsDF, left_on='id', right_on='id')

In [44]:
merge = merge.drop_duplicates(subset=['id'])

In [46]:
writepath = "/Users/pradneshkolluru/OneDrive - Duke University/RARE_BASS/Standardized"

In [48]:
for row,items in merge.iterrows():
    raw_filename = os.path.join(readpath, items["filename"])
    raw = nib.load(raw_filename)
    scaledRaw = raw.get_fdata() * items["mRatio"]
    lab_filename = os.path.join(readpath, items["labelFile"])
    lab = nib.load(lab_filename)
    labels = lab.get_fdata()
    lab_mask = np.array((np.ma.array(labels) > 0)*1)
    brain_bin = np.multiply(lab_mask, scaledRaw)    
    img = nib.Nifti1Image(brain_bin, lab.affine)
    nib.save(img, writepath + "/" + "standardized_" + items["filename"])